In [3]:
import pandas as pd
import clr
clr.AddReference('C:\\Program Files\\Siemens\\Automation\\Portal V15_1\PublicAPI\\V15.1\\Siemens.Engineering.dll')
from System.IO import DirectoryInfo
import Siemens.Engineering as tia
import Siemens.Engineering.HW.Features as hwf


In [2]:
project_path = DirectoryInfo ('C:\\Jonas\\TIA')
project_name = 'PythonTest'

#Starting TIA
print ('Starting TIA with UI')
mytia = tia.TiaPortal(tia.TiaPortalMode.WithUserInterface)
try:
    myproject = mytia.Projects.Create(project_path, project_name)
except Exception as e:
    print (e)

Starting TIA with UI


In [4]:
processes = tia.TiaPortal.GetProcesses() # Making a list of all running processes
process = processes[0]                   # Just taking the first process as an example
mytia = process.Attach()
myproject = mytia.Projects[0]

In [10]:
#mytia.Dispose()


In [7]:
df = pd.read_excel('./resources/template.xlsx', sheet_name='Sheet1')
df = df.replace({pd.np.nan: None}) # Needed as "HMI name2 must be None"
#df


In [8]:
node_names1 = df['Name1']
node_names2 = df['Name2']
nodes = df['Main station']
node_ip = df['IP']





In [11]:
stations = []
for i in range(len(nodes)):
    stations.append(myproject.Devices.CreateWithItem(nodes[i], node_names1[i], node_names2[i]))
    

In [26]:
# adding IO
for station in stations:
    io = []
    for i in range(1,6):
        if df[f'IO.{i}'][stations.index(station)] is not (None):
            if (station.DeviceItems[0].CanPlugNew(df[f'IO.{i}'][stations.index(station)], f'IO{i}', (i))) is True:
                (station.DeviceItems[0].PlugNew(df[f'IO.{i}'][stations.index(station)], f'IO{i}', (i)))
            elif (station.DeviceItems[0].CanPlugNew(df[f'IO.{i}'][stations.index(station)], f'IO{i}', (i+1))) is True:
                (station.DeviceItems[0].PlugNew(df[f'IO.{i}'][stations.index(station)], f'IO{i}', (i+1)))

In [6]:
# Creating network interfaces and setting IPs
n_interfaces = []
for station in stations:
    device_item_aggregation = station.DeviceItems[1].DeviceItems
    for deviceitem in device_item_aggregation:
        network_service = tia.IEngineeringServiceProvider(deviceitem).GetService[hwf.NetworkInterface]()
        if type(network_service) is hwf.NetworkInterface:
            n_interfaces.append(network_service)
            network_service.Nodes[0].SetAttribute('Address',node_ip[stations.index(station)])


In [7]:
# Creating subnet and IO system
for n in n_interfaces:
    if n_interfaces.index(n) == 0:
        subnet = n_interfaces[0].Nodes[0].CreateAndConnectToSubnet("Profinet")
        ioSystem = n_interfaces[0].IoControllers[0].CreateIoSystem("PNIO");
    else:
        n_interfaces[n_interfaces.index(n)].Nodes[0].ConnectToSubnet(subnet)
        if (n_interfaces[n_interfaces.index(n)].IoConnectors.Count) >> 0:
            n_interfaces[n_interfaces.index(n)].IoConnectors[0].ConnectToIoSystem(ioSystem);

